# Extracting images for each embryo

Even though the FOV contains several embryos, there is a considerable amount of background pixels that can be ignored.
Slicing the raw image into individual embryos offers significant memory reduction (average 80% for the images processed so far) and allows for easier processing downstream, since individual movies easily fit the RAM of a regular PC.

The first step is to calculate the boundaries of each embryo.
The first 10 frames of the movie are used to calculate these boundaries.
Using a low threshold (triangle method) we binarize the image and label each one of the embryos.

Once we have the boundaries, we open the entire raw data as a memmap, and extract each embryo, splitting channels 1 (active) and 2 (structural).

Embryos will be numbered in column-major order: top left corner receives 1, then second embryo is the one immediately below, etc.

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
from pathlib import Path

import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

from snazzy_processing import slice_img

If data was collected as an nd2 movie, it must first be converted to a tif file, and that tif file should be used in the other cells of this jupyter notebook.

In [ ]:
nd2_path = "/home/cdp58/Documents/raw_images/pasnascope/20240611_25C.nd2"
img_path = "/home/cdp58/Documents/raw_images/pasnascope/20240611.tif"
should_convert = False

if should_convert:
    slice_img.save_as_tiff(nd2_path, img_path)
else:
    print("If you intended to convert the file, change `should_convert` to True.")

Visualize the boxes used to extract each movie.
The numbers displayed will be used to name each embryo.

In [ ]:
img = slice_img.get_first_image_from_mmap(img_path)

coords = slice_img.calculate_slice_coordinates(img_path, n_cols=4, thres_adjust=0)
boundaries = coords.values()

rect_coords = [slice_img.boundary_to_rect_coords(b, img.shape) for b in boundaries]
recs = [Rectangle((x, y), w, h) for (y, x, w, h) in rect_coords]

centroids = {
    i: ((x0 + x1) // 2, (y0 + y1) // 2) for i, (x0, x1, y0, y1) in coords.items()
}
h = img.shape[0]
w = img.shape[1]

fig, ax = plt.subplots()
fig.canvas.header_visible = False
fig.canvas.resizable = False
ax.set_axis_off()
ax.imshow(img)

for i, (y, x) in centroids.items():
    ax.text(x, y, str(i))

pc = PatchCollection(recs, color="red", alpha=0.7, linewidth=1, facecolor="none")
ax.add_collection(pc)

plt.show()

After inspecting the slices that will be generated, change `should_process` to `True`, so that the image is actually processed.
Make sure that `experiment_name` corresponds to the image you are parsing, since it will be used to name the folder associated to this image.
The code assumes channel 1 is the active channel (GCaMP6) and channel 2 is the structural channel.
This can be changed by passing `active_ch=2` to `slice_img.cut_movies()`. 
Regardless of the option you choose, for the output files, ch1 will be the active channel and ch2 will be the structural channel.

In [ ]:
experiment_name = "20240515"
root_dir = Path.cwd().parent
dest = root_dir.joinpath("data", experiment_name, "embs")
should_process = False

if should_process:
    dest.mkdir(parents=True, exist_ok=True)
    slice_img.cut_movies(coords, img_path, dest, active_ch=2, overwrite=False)
else:
    print("If you intended to parse the data, change `should_convert` to True.")